In [1]:
from edbo.plus.optimizer_botorch import EDBOplus
from edbo.edbo.feature_utils import reaction_space

C:\Anaconda3\envs\edbo\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#### This is a tutorial that covers the basics for running EDBO+: from designing a combinatorial space to running the Bayesian Optimizer.

## 1. Creating a search scope using EDBO+.

##### To run EDBO+ we need to first create a reaction scope (search space) in a .csv format with all the possible combinations that you want to consider for our optimization. 
##### You can "manually" create a reaction scope using any spreadsheet editor (such as Excel, Libreoffice Calc, ...) but we have also created a tool to help you generating combinatorial search spaces. 
##### For instance, lets say that we want to consider 3 solvents ($\bf{THF}$, $\bf{Toluene}$, $\bf{DMSO}$), 4 temperatures ($\bf{-10}$, $\bf{0}$, $\bf{10}$, $\bf{25}$) and 3 concentration levels ($\bf{0.1}$, $\bf{0.2}$, $\bf{1.0}$). We can introduce these in the EDBO+ scope generator in a dictionary form as follows:

In [12]:
### 1_dataprocess 에서 저장한 dataset 파일을 불러와 BO를 진행. 적절한 변수를 이용해 필요한 모든 dataset을 불러옵니다. 

import os
import pandas as pd
from edbo.edbo.utils import Data


### 결과 폴더 지정. 실제 csv 파일이 있는 경로를 지정합니다. FILE_PREFIX는 추후 저장할 파일명에 사용됨
FOLDER = './result'
FILE_PREFIX = 'my_optimization'

### 적절한 csv 파일명과 변수 명을 설정해주세요. 
sample = Data(pd.read_csv(os.path.join(FOLDER, 'sample.csv')))   
### 파일이 여러개일 경우 아래와 같은 포맷으로 파일 로드
#solvent = Data(pd.read_csv(os.path.join(FOLDER, 'solvent.csv')))   



# Use Data.drop method to drop descriptors containing some unwanted keywords
#for data in [azadicarbs, phosphines, solvents]:
#    data.drop(['file_name', 'entry', 'vibration', 'correlation', 'Rydberg', 
#               'correction', 'atom_number', 'E-M_angle', 'MEAN', 'MAXG', 
#               'STDEV'])

In [13]:
sample.data

can  sample_E_boltz  sample_ES_root_dipole_boltz  \
0  C(C1C(C(C(C(O1)O)O)O)O)O     -686.483139                     3.655941   
1               c1ccc(F)cc1     -331.128965                     1.302600   

   sample_ES_root_electronic_spatial_extent_boltz  \
0                                      1945.71727   
1                                       647.67130   

   sample_ES_root_molar_volume_boltz  sample_E_scf_boltz  \
0                        1424.188353         -686.687703   
1                         945.523000         -331.226739   

   sample_E_thermal_correction_boltz  sample_E_zpe_boltz  sample_G_boltz  \
0                           0.212099         -686.494718     -686.531304   
1                           0.098046         -331.134091     -331.162907   

   sample_G_thermal_correction_boltz  ...  sample_electronegativity_minE  \
0                           0.163934  ...                       0.095995   
1                           0.064104  ...                       0.132065   

   sample_electronic_spatial_extent_minE  sample_hardness_minE  \
0                              1970.1944              0.169285   
1                               647.7917              0.122445   

   sample_homo_energy_minE  sample_lumo_energy_minE  sample_molar_mass_minE  \
0                 -0.26528                  0.07329                180.1572   
1                 -0.25451                 -0.00962                 96.1039   

   sample_molar_volume_minE  sample_multiplicity_minE  \
0                  1294.255                       1.0   
1                   785.717                       1.0   

   sample_number_of_atoms_minE  sample_zero_point_correction_minE  
0                         24.0                           0.200557  
1                         12.0                           0.092920  

[2 rows x 111 columns]

In [15]:
### 위에서 읽어들인 DFT 파일을 이용해 Bayesian Optimization Scope 을 생성합니다. 
### 아래 components와 dft 설정에 위에서 load한 data를 기재해주세요. 

components = {'sample':'DFT',                             # DFT descriptors
              #'phosphine':'DFT',                                    # DFT descriptors
              #'solvent':'DFT',                                      # DFT descriptors
              'substrate_concentration':[0.05, 0.10],   # Discrete grid of concentrations
              'azadicarb_equiv':[1.1, 1.3, 1.5],          # Discrete grid of equiv.
              'phos_equiv':[1.5, 1.7, 1.9],               # Discrete grid of equiv.
              'temperature':[5, 15, 25]}                    # Discrete grid of temperatures

### Encodings - if not specified EDBO will automatically use OHE

encoding = {'substrate_concentration':'numeric',                    # Numerical encoding
            'azadicarb_equiv':'numeric',                            # Numerical encoding
            'phos_equiv':'numeric',                                 # Numerical encoding
            'temperature':'numeric'}                                # Numerical encoding

### External descriptor matrices override specified encoding

dft = { 'sample' : sample.data
     #   'azadicarboxylate':azadicarbs.data,                          # Unprocessed descriptor DataFrame
     #  'phosphine':phosphines.data,                                 # Unprocessed descriptor DataFrame
     #  'solvent':solvents.data}                                     # Unprocessed descriptor DataFrame
      }
df_scope = reaction_space(components, 
                    encoding=encoding,
                    descriptor_matrices=dft, clean=True, decorrelate=True)
df_scope.data.to_csv(os.path.join(FOLDER, f'{FILE_PREFIX}_scope.csv'), index=False)

In [16]:
df_scope.data

sample_E_boltz  substrate_concentration  azadicarb_equiv  phos_equiv  \
0               0.0                      0.0              0.0         0.0   
1               0.0                      0.0              0.0         0.0   
2               0.0                      0.0              0.0         0.0   
3               0.0                      0.0              0.0         0.5   
4               0.0                      0.0              0.0         0.5   
..              ...                      ...              ...         ...   
103             1.0                      1.0              1.0         0.5   
104             1.0                      1.0              1.0         0.5   
105             1.0                      1.0              1.0         1.0   
106             1.0                      1.0              1.0         1.0   
107             1.0                      1.0              1.0         1.0   

     temperature  
0            0.0  
1            0.5  
2            1.0  
3            0.0  
4            0.5  
..           ...  
103          0.5  
104          1.0  
105          0.0  
106          0.5  
107          1.0  

[108 rows x 5 columns]

##### We can always load/read the previously generated reaction scope using any spreadsheet editor but in this case we will use Pandas for that:

In [18]:
import pandas as pd
df_scope = pd.read_csv(os.path.join(FOLDER, f'{FILE_PREFIX}_scope.csv'))  # Load csv file.

##### Now we can check the number of combinations in the reaction scope:

In [19]:
n_combinations = len(df_scope)
print(f"Your reaction scope has {n_combinations} combinations.")

Your reaction scope has 108 combinations.


##### Of course, this is a very small reaction scope  as it is meant to be a toy model to demonstrate how EDBO+ works.

## 2. First steps, initializing EDBO+ (in absence of training data).

##### We are going to execute EDBO+ to suggest some initial samples. 
##### Since we have not collected any experimental data (observations) yet, EDBO+ will suggest a set of initial experiments based on an feature space sampling method, in this case the CVT sampling method (see:http://kmh-lanl.hansonhub.com/uncertainty/meetings/gunz03vgr.pdf).
##### In this example the $\bf{objective}$ is to maximize the reaction $\bf{yield}$ and $\bf{enantioselectivity}$ but at the same time we want to minimize the amount one of a given $\bf{side~product}$ in this reaction. We also need to introduce the name of the csv file containing our reaction scope (in our case this was $\bf{my\_optimization.csv}$). Now we can execute the algorithm using the $\bf{run}$ command in EDBOplus:

In [20]:
df_scope

sample_E_boltz  substrate_concentration  azadicarb_equiv  phos_equiv  \
0               0.0                      0.0              0.0         0.0   
1               0.0                      0.0              0.0         0.0   
2               0.0                      0.0              0.0         0.0   
3               0.0                      0.0              0.0         0.5   
4               0.0                      0.0              0.0         0.5   
..              ...                      ...              ...         ...   
103             1.0                      1.0              1.0         0.5   
104             1.0                      1.0              1.0         0.5   
105             1.0                      1.0              1.0         1.0   
106             1.0                      1.0              1.0         1.0   
107             1.0                      1.0              1.0         1.0   

     temperature  
0            0.0  
1            0.5  
2            1.0  
3            0.0  
4            0.5  
..           ...  
103          0.5  
104          1.0  
105          0.0  
106          0.5  
107          1.0  

[108 rows x 5 columns]

In [33]:
### objective 설정 필요. 실험에서 얻을 output 조건과 maximize or minimize할 것인지를 지정. 
EDBOplus().run(
    directory=FOLDER,
    filename=f'{FILE_PREFIX}_scope.csv',  # Previously generated scope.
    objectives=['yield', 'ee', 'side_product'],  # Objectives to be optimized.
    objective_mode=['max', 'max', 'min'],  # Maximize yield and ee but minimize side_product.
    batch=3,  # Number of experiments in parallel that we want to perform in this round.
    columns_features='all', # features to be included in the model.
    init_sampling_method='cvtsampling'  # initialization method.
)

The scope was already generated, please insert at least one experimental observation value and then press run.


sample_E_boltz  substrate_concentration  azadicarb_equiv  phos_equiv  \
0               0.0                      1.0              0.5         1.0   
1               1.0                      0.0              0.5         0.5   
2               0.0                      1.0              0.5         0.0   
3               1.0                      0.0              1.0         1.0   
4               1.0                      0.0              1.0         0.5   
..              ...                      ...              ...         ...   
103             0.0                      1.0              0.0         0.5   
104             0.0                      1.0              0.0         0.5   
105             0.0                      1.0              0.0         0.5   
106             0.0                      1.0              0.0         0.0   
107             1.0                      1.0              1.0         1.0   

     temperature    yield       ee side_product  priority  
0            0.0  PENDING  PENDING      PENDING         1  
1            1.0  PENDING  PENDING      PENDING         1  
2            0.0  PENDING  PENDING      PENDING         1  
3            0.0  PENDING  PENDING      PENDING         0  
4            1.0  PENDING  PENDING      PENDING         0  
..           ...      ...      ...          ...       ...  
103          1.0  PENDING  PENDING      PENDING         0  
104          0.5  PENDING  PENDING      PENDING         0  
105          0.0  PENDING  PENDING      PENDING         0  
106          1.0  PENDING  PENDING      PENDING         0  
107          1.0  PENDING  PENDING      PENDING         0  

[108 rows x 9 columns]

##### EDBO+ has created a column for each objective and added $\bf{PENDING}$ values to all of them so you can track the experiments that you have been collecting during the optimization campaign.
##### We can also see that EDBO+ has created a new $\bf{priority}$ column. This column is used to distinguish between high and low priority samples. The top entries (with $\bf{priority=1}$) highlight the next suggested samples.

##### We can check now the first 5 experiments in the scope by reading the $\bf{my\_optimization.csv}$ file:

In [22]:
df_edbo = pd.read_csv(os.path.join(FOLDER, f'{FILE_PREFIX}_scope.csv'))
df_edbo.head(5)

sample_E_boltz  substrate_concentration  azadicarb_equiv  phos_equiv  \
0             0.0                      1.0              0.5         1.0   
1             1.0                      0.0              0.5         0.5   
2             0.0                      1.0              0.5         0.0   
3             1.0                      0.0              1.0         1.0   
4             1.0                      0.0              1.0         0.5   

   temperature    yield       ee side_product  priority  
0          0.0  PENDING  PENDING      PENDING         1  
1          1.0  PENDING  PENDING      PENDING         1  
2          0.0  PENDING  PENDING      PENDING         1  
3          0.0  PENDING  PENDING      PENDING         0  
4          1.0  PENDING  PENDING      PENDING         0

## 3. Adding training data in EDBO+.

##### Note: We will use Python and Pandas to add new training data in this example. But you can always edit and add new data into the '.csv' file using any spreedsheet editor (such as Excel, Libreoffice Calc, ...) if that's more convinient for you.

##### Let's open again the $\bf{my\_optimization.csv}$ file we generated before:

In [8]:
## 아래와 같이 코드 상에서 값을 지정하거나 혹은 csv 파일에 직접 값을 기재하여 사용합니다. 

df_edbo = pd.read_csv(os.path.join(FOLDER, f'{FILE_PREFIX}_scope.csv')
df_edbo.head(5)

sample_E_boltz  substrate_concentration  azadicarb_equiv  phos_equiv  \
0             0.0                      1.0              0.5         1.0   
1             1.0                      0.0              0.5         0.5   
2             0.0                      1.0              0.5         0.0   
3             1.0                      0.0              1.0         1.0   
4             1.0                      0.0              1.0         0.5   

   temperature    yield       ee side_product  priority  
0          0.0  PENDING  PENDING      PENDING         1  
1          1.0  PENDING  PENDING      PENDING         1  
2          0.0  PENDING  PENDING      PENDING         1  
3          0.0  PENDING  PENDING      PENDING         0  
4          1.0  PENDING  PENDING      PENDING         0

##### We can fill the first out entry in the previous dataframe with the "observed" values using Pandas:

In [23]:
df_edbo.loc[0, 'yield'] = 20.5
df_edbo.loc[0, 'ee'] = 40
df_edbo.loc[0, 'side_product'] = 0.1

##### We can check that we have filled out the first entry with our "observed data":

In [24]:
df_edbo.head(5)

sample_E_boltz  substrate_concentration  azadicarb_equiv  phos_equiv  \
0             0.0                      1.0              0.5         1.0   
1             1.0                      0.0              0.5         0.5   
2             0.0                      1.0              0.5         0.0   
3             1.0                      0.0              1.0         1.0   
4             1.0                      0.0              1.0         0.5   

   temperature    yield       ee side_product  priority  
0          0.0     20.5       40          0.1         1  
1          1.0  PENDING  PENDING      PENDING         1  
2          0.0  PENDING  PENDING      PENDING         1  
3          0.0  PENDING  PENDING      PENDING         0  
4          1.0  PENDING  PENDING      PENDING         0

##### We can also fill out the second entry with their corresponding "observations":

In [25]:
df_edbo.loc[1, 'yield'] = 50.3
df_edbo.loc[1, 'ee'] = 10
df_edbo.loc[1, 'side_product'] = 0.2

In [26]:
df_edbo.head(5)

sample_E_boltz  substrate_concentration  azadicarb_equiv  phos_equiv  \
0             0.0                      1.0              0.5         1.0   
1             1.0                      0.0              0.5         0.5   
2             0.0                      1.0              0.5         0.0   
3             1.0                      0.0              1.0         1.0   
4             1.0                      0.0              1.0         0.5   

   temperature    yield       ee side_product  priority  
0          0.0     20.5       40          0.1         1  
1          1.0     50.3       10          0.2         1  
2          0.0  PENDING  PENDING      PENDING         1  
3          0.0  PENDING  PENDING      PENDING         0  
4          1.0  PENDING  PENDING      PENDING         0

##### Now we can save our dataset as $\bf{my\_optimization\_round0.csv}$:

In [27]:
df_edbo.to_csv(os.path.join(FOLDER, f'{FILE_PREFIX}_round0.csv'), index=False)

## 4. Running EDBO+ with training data.

##### First let's check our previous data (which include some $\bf{yield}$, $\bf{ee}$ and $\bf{side\_product}$ observations, which will be used to train the model):

In [34]:
df_edbo_round0 = pd.read_csv(os.path.join(FOLDER, f'{FILE_PREFIX}_round0.csv'))
df_edbo_round0.head(5)

sample_E_boltz  substrate_concentration  azadicarb_equiv  phos_equiv  \
0             0.0                      1.0              0.5         1.0   
1             1.0                      0.0              0.5         0.5   
2             0.0                      1.0              0.5         0.0   
3             1.0                      0.0              1.0         1.0   
4             1.0                      0.0              1.0         0.5   

   temperature    yield       ee side_product  priority  
0          0.0     20.5       40          0.1         1  
1          1.0     50.3       10          0.2         1  
2          0.0  PENDING  PENDING      PENDING         1  
3          0.0  PENDING  PENDING      PENDING         0  
4          1.0  PENDING  PENDING      PENDING         0

##### Now that we have introduced some "observations" in our $\bf{my\_optimization\_round0.csv}$ file, we can execute EDBO+ to suggest samples using these "observations" as training data.

In [36]:
EDBOplus().run(
    directory=FOLDER,
    filename=f'{FILE_PREFIX}_round0.csv',  # Previous scope (including observations).
    objectives=['yield', 'ee', 'side_product'],  # Objectives to be optimized.
    objective_mode=['max', 'max', 'min'],  # Maximize yield and ee but minimize side_product.
    batch=3,  # Number of experiments in parallel that we want to perform in this round.
    columns_features='all', # features to be included in the model.
    init_sampling_method='cvtsampling'  # initialization method.
)

Using EHVI acquisition function.
Using hyperparameters optimized for continuous variables.
Using hyperparameters optimized for continuous variables.
Using hyperparameters optimized for continuous variables.
Number of QMC samples using SobolQMCNormalSampler sampler: 512
Acquisition function optimized.
Predictions obtained and expected improvement obtained.


sample_E_boltz  substrate_concentration  azadicarb_equiv  phos_equiv  \
85              0.0                      1.0              1.0         1.0   
83              0.0                      1.0              0.0         0.0   
74              0.0                      0.0              0.0         1.0   
107             1.0                      1.0              1.0         1.0   
30              1.0                      1.0              1.0         1.0   
..              ...                      ...              ...         ...   
80              0.0                      0.0              0.0         0.0   
56              0.0                      0.0              0.0         0.0   
13              0.0                      0.0              0.0         0.0   
1               1.0                      0.0              0.5         0.5   
0               0.0                      1.0              0.5         1.0   

     temperature    yield       ee side_product  priority  
85           0.5  PENDING  PENDING      PENDING       1.0  
83           0.0  PENDING  PENDING      PENDING       1.0  
74           1.0  PENDING  PENDING      PENDING       1.0  
107          1.0  PENDING  PENDING      PENDING       0.0  
30           0.5  PENDING  PENDING      PENDING       0.0  
..           ...      ...      ...          ...       ...  
80           1.0  PENDING  PENDING      PENDING       0.0  
56           0.5  PENDING  PENDING      PENDING       0.0  
13           0.0  PENDING  PENDING      PENDING       0.0  
1            1.0     50.3       10          0.2      -1.0  
0            0.0     20.5       40          0.1      -1.0  

[108 rows x 9 columns]

##### Again the samples suggested by EDBO+ have $\bf{priority = +1}$. In addition, we asign $\bf{priority = -1}$ to the experiments that we have already run (these are at the bottom of the dataset).

## Extra: Accessing the model predictions.

##### Each time that EDBO+ is executed with training data it will generate a .csv file with the predictions for the entire scope (including the 'untested' samples).
##### In the previous example, by running EDBO+ with training data, we generated two files: $\bf{my\_optimization\_round0.csv}$ and a second file with the predictions $\bf{pred\_my\_optimization\_round0.csv}$.
##### Let's have a look to the predictions file using Pandas:

In [16]:
df_predictions_round0 = pd.read_csv('pred_my_optimization_round0.csv')
df_predictions_round0.style.background_gradient(subset=['priority'], cmap='plasma')

##### In the previous dataset we can access the model predictions ($\bf{predicted\_mean}$ and $\bf{predicted\_variance}$ columns) but also to the expected improvement values (in the $\bf{expected\_improvement}$ columns) for each objective ($\bf{yield}$, $\bf{ee}$ and $\bf{side\_product}$.).